In [1]:
import gym
import gym_sokoban
import time
import datetime as dt
import matplotlib.pyplot as plt
import math
from collections import deque
import random
import numpy as np
import pickle
import sys
import tqdm
import os
from collections import namedtuple,defaultdict,deque 

In [2]:
chapter = 0
level = 1


env_name = 'Sokoban-v1'
env = gym.make(env_name)
ACTION_LOOKUP = env.unwrapped.get_action_lookup()
env.unwrapped.set_level(chapter,level)
env.seed(0)
env.reset()
print("Created environment: {}".format(env_name))


#create Results/Chapter [chapter]/Level [level] folder
if not os.path.exists('Results/Chapter '+str(chapter)+'/Level '+str(level)):
    os.makedirs('Results/Chapter '+str(chapter)+'/Level '+str(level))

Created environment: Sokoban-v1


In [3]:
while True:
    env.render()
    time.sleep(1/60)

333300333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333003333
333300333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333003333


KeyboardInterrupt: 

In [3]:
def map( x,  in_min,  in_max,  out_min,  out_max) :
  return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min;

def greedy_policy(V,s,eps_comp=1e-8):
    #.9 prob of greedy action
    #.1 prob of random action

    if s not in V:
        V[s] = np.zeros(env.action_space.n)
    r_choice = map(eps_comp,0,1,0.1,.4)
    if np.random.random() < r_choice:
        return np.random.choice(np.arange(env.action_space.n))
    else:
        max_val = np.max(V[s])
         #find all actions that have the max value and choose one at random
        max_actions = np.argwhere(V[s] == max_val).flatten()
        return np.random.choice(max_actions)

In [4]:
EVERY_VISIT_MC = False
EPISODES = 5000

V = {}
total_returns = {}
N = {}
for episode in tqdm.tqdm(range(EPISODES)):
    
    visited = []
    env.reset()
    state = env.unwrapped.serialize_state()
    done = False
    for t in range(240):
        action_time = time.time()
        if done:
            break
        action = greedy_policy(V,state,eps_comp=episode/EPISODES)
        next_state, reward, done, info = env.step(action)
        next_state = env.unwrapped.serialize_state()
        if EVERY_VISIT_MC or (not EVERY_VISIT_MC and state not in visited):
            if not EVERY_VISIT_MC:
                visited.append(state)
            if state not in total_returns:
                total_returns[state] = np.zeros(env.action_space.n)
            for _state in total_returns:
                total_returns[_state][action] += reward
            if state not in N:
                N[state] = np.zeros(env.action_space.n)
            
            N[state][action] += 1
            V[state][action] = (total_returns[state][action] / N[state][action])
        state = next_state
    if episode % 1000 == 0:
        fname = 'Results/Chapter '+str(chapter)+'/Level '+str(level)+'/MC_'+('every' if EVERY_VISIT_MC else 'first')+'_'+str(EPISODES)+'_episodes_temp.bin'
        with open(fname, 'wb') as handle:
            pickle.dump(V, handle, protocol=pickle.HIGHEST_PROTOCOL)


            
#save the value function
fname = 'Results/Chapter '+str(chapter)+'/Level '+str(level)+'/MC_'+('every' if EVERY_VISIT_MC else 'first')+'_'+str(EPISODES)+'_episodes.bin'
with open(fname, 'wb') as handle:
    pickle.dump(V, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

  0%|          | 0/5000 [00:00<?, ?it/s]

100%|██████████| 5000/5000 [09:58<00:00,  8.36it/s]


In [7]:
fname = 'Results/Chapter '+str(chapter)+'/Level '+str(level)+'/MC_'+('every' if EVERY_VISIT_MC else 'first')+'_'+str(EPISODES)+'_episodes.bin'
with open(fname, 'wb') as handle:
    pickle.dump(V, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
N.values()

dict_values([array([33664.,  4090.,  4163.,  4005.,  4078.]), array([3565.,  527.,  539.,  595.,  475.]), array([3166.,  440.,  481.,  479.,  421.]), array([31197.,  3660.,  3737.,  3812.,  3749.]), array([15017.,  1864.,  1883.,  1965.,  1845.]), array([2881.,  396.,  419.,  416.,  382.]), array([10838.,  1310.,  1373.,  1356.,  1288.]), array([12721.,  1414.,  1443.,  1519.,  1446.]), array([11209.,  1228.,  1273.,  1339.,  1207.]), array([2762.,  375.,  356.,  411.,  320.]), array([2688.,  380.,  362.,  336.,  351.]), array([2624.,  378.,  336.,  350.,  353.]), array([2603.,  355.,  337.,  348.,  385.]), array([5263.,  598.,  591.,  619.,  592.]), array([4519.,  496.,  488.,  527.,  475.]), array([3931.,  431.,  441.,  462.,  388.]), array([432.,  56.,  54.,  65.,  53.]), array([69., 12.,  9., 18., 16.]), array([48.,  7.,  5.,  9.,  8.]), array([11.,  4.,  1.,  1.,  1.]), array([3923.,  408.,  431.,  455.,  405.]), array([3697.,  370.,  361.,  372.,  383.]), array([3328.,  345.,  36

In [6]:
V

{'3333333333333300000000000033000035040003000000000000331730080400530000000000003333333333333300000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000': array([17.64170883, 13.24942274, 13.31457146, 15.02842247, 14.3081128 ]),
 '333333333333330000000000003310003504000300000000000033073008040053000000000000333333333333330000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [5]:

for s in V:
    argmax = np.argmax(V[s])
    max_val = np.max(V[s])
    s = s[26:40] +'\n'+ s[26+26:40+26]
    print("{} Action: {} Value: {}".format(s,ACTION_LOOKUP[argmax],max_val))

37010008004053
33333333333333 Action: right Value: 15.241995743386466
37100008004053
33333333333333 Action: right Value: 48.71248007774352
31000008004053
33333333333333 Action: right Value: 90.4073519855561
30100008004053
33333333333333 Action: right Value: 106.11448093219933
37001008004053
33333333333333 Action: right Value: 16.89032681940636
37000108004053
33333333333333 Action: right Value: 17.86513699607495
37000018004053
33333333333333 Action: right Value: 18.75458136924732
30010008004053
33333333333333 Action: right Value: 124.62990749999516
30001008004053
33333333333333 Action: right Value: 141.23490571428002
30000108004053
33333333333333 Action: right Value: 167.03844406778987
30000018004053
33333333333333 Action: right Value: 195.31046245058505
30000010004053
33333333333333 Action: right Value: 245.86519900496526
30000001004053
33333333333333 Action: right Value: 295.95365868262286
30000100004053
33333333333333 Action: right Value: 3851.9134999998278
30001000004053
33333333333

In [11]:
def greedy_policy_pi(V,s):
    #.9 prob of greedy action
    #.1 prob of random action

    if s not in V:
        V[s] = np.zeros(env.action_space.n)
    r_choice = .3
    if np.random.random() < r_choice:
        return np.random.choice(np.arange(env.action_space.n))
    else:
        max_val = np.max(V[s])
         #find all actions that have the max value and choose one at random
        max_actions = np.argwhere(V[s] == max_val).flatten()
        return np.random.choice(max_actions)


In [12]:
_ = env.reset()
state = env.unwrapped.serialize_state()


In [14]:

for i in range(100):
    time.sleep(1)
    _ = env.reset()
    state = env.unwrapped.serialize_state()
    st_time = time.time()
    done = False
    t= 0
    last_reward = 0
    while True:
        if time.time() - st_time < .1:
            env.render()
            time.sleep(1/60)
            continue
        st_time = time.time()
        action_time = time.time()
        action = greedy_policy_pi(V,state)
        next_state, reward, done, info = env.step(action)
        next_state = env.unwrapped.serialize_state()
        if reward > last_reward:
            print("Action: {} Reward: {}".format(ACTION_LOOKUP[action],round(reward,3)),end="\n")

        last_reward = reward
        t += 1

        if done or t > 200:
            env.render()
            break
        env.render()

Exception ignored in: <function SimpleImageViewer.__del__ at 0x0000021070A3BEE0>
Traceback (most recent call last):
  File "d:\2023-2024\RNN\Sokoban\SokobanEnv\lib\site-packages\gym\envs\classic_control\rendering.py", line 458, in __del__
    self.close()
  File "d:\2023-2024\RNN\Sokoban\SokobanEnv\lib\site-packages\gym\envs\classic_control\rendering.py", line 454, in close
    self.window.close()
  File "d:\2023-2024\RNN\Sokoban\SokobanEnv\lib\site-packages\pyglet\window\win32\__init__.py", line 299, in close
    super(Win32Window, self).close()
  File "d:\2023-2024\RNN\Sokoban\SokobanEnv\lib\site-packages\pyglet\window\__init__.py", line 658, in close
    app.windows.remove(self)
  File "C:\Users\duong\AppData\Local\Programs\Python\Python39\lib\_weakrefset.py", line 114, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x000002106DD7F4A0; to 'Win32Window' at 0x000002106DB2C2B0>


Action: right Reward: 2.998
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: extinguish Reward: 2.95
Action: right Reward: 2.998
Action: extinguish Reward: 2.95
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: left Reward: 2.998
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: down Reward: 2.948
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: left Reward: 2.998
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: left Reward: 2.998
Action: up Reward: 2.948
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: up Reward: 2.948
Action: up Reward: 2.948
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: up Reward: 2.948
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: right Reward: 2.998
Action: right Reward: 2.998


In [12]:
ACTION_LOOKUP

{0: 'right', 1: 'up', 2: 'down', 3: 'left', 4: 'extinguish'}

In [4]:

action = 4
next_state, reward, done, info = env.step(action)
env.render()
print("Action: {} Reward: {}".format(ACTION_LOOKUP[action],reward))

333300333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333003333
333300333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333003333
Action: extinguish Reward: 2.95
